<a href="https://colab.research.google.com/github/darshanvjani/Extensive-Vision-AI-Program-EVAI6-/blob/main/4_Basics_of_Architecture/MNIST_custom_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,16,3,bias=False), #(28,28,1) -> (26,26,8) | RF=2
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.07),

            nn.Conv2d(16,16,3,bias=False), #(26,26,8) -> (24,24,16) | RF=4
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.07),

            nn.Conv2d(16,16,3,bias=False),  #(24,24,16) -> (22,22,16) | RF=6
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.07)
        )
        self.trans1 = nn.Sequential(
            nn.Conv2d(16,8,1,bias=False), #(22,22,16) -> (22,22,8) | RF=6
            nn.ReLU(),
            nn.MaxPool2d(2) #(22,22,16) -> (11,11,8) | RF=12
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(8,16,3,bias=False), #(11,11,8) -> (9,9,16) | RF=14
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.07),

            nn.Conv2d(16,16,3,bias=False),  #(9,9,16) -> (7,7,16) | RF=16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.07),

            nn.Conv2d(16,16,3,bias=False),  #(7,7,16) -> (5,5,16) | RF=18
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.07)
        )
        self.trans2 = nn.Sequential(
            nn.Conv2d(16,10,1,bias=False), #(5,5,16) -> (5,5,10) | RF=18
            nn.ReLU(),
            nn.AvgPool2d(5) #(5,5,10) -> (1,1,10) | RF=22
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.trans1(x)
        x = self.conv2(x)
        x = self.trans2(x)

        x = x.reshape(-1,10)
        return F.log_softmax(x)

In [19]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,304
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
        Dropout2d-12           [-1, 16, 22, 22]               0
           Conv2d-13            [-1, 8, 22, 22]             128
             ReLU-14            [-1, 8,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [8]:
img,label = next(iter(train_loader))
img.size()

torch.Size([128, 1, 28, 28])

In [20]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

9853/10000 (99%) (GAP, ANTMAN) 

9823/10000 (98%) (GAP, ANTMAN, DROPOUT)

9938/10000 (99%) (GAP, ANTMAN, DROPOUT, BATCHNORM)


In [21]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.2070837765932083 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.29it/s]



Test set: Average loss: 0.0732, Accuracy: 9806/10000 (98%)



loss=0.13383884727954865 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.61it/s]



Test set: Average loss: 0.0449, Accuracy: 9863/10000 (99%)



loss=0.05832888185977936 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.03it/s]


...


Test set: Average loss: 0.0211, Accuracy: 9944/10000 (99%)



loss=0.06752670556306839 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.31it/s]



Test set: Average loss: 0.0183, Accuracy: 9945/10000 (99%)



loss=0.054910313338041306 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.47it/s]



Test set: Average loss: 0.0185, Accuracy: 9945/10000 (99%)

